In [1]:
import xarray as xr


info_ds = xr.open_dataset("/lustre/geocean/WORK/users/faugeree/GreenSurge/Mar_del_Norte_v2/Eje_01/Data_1_GFD_lib_info.nc")
info_ds

<xarray.Dataset> Size: 149kB
Dimensions:         (M: 3572, N: 4218, Node: 597, teselas: 933, NN: 3,
                     Ndir: 24, date_ref: 6)
Coordinates:
  * date_ref        (date_ref) int32 24B 2022 1 1 0 0 0
  * M               (M) int32 14kB 1 2 3 4 5 6 ... 3567 3568 3569 3570 3571 3572
  * N               (N) int32 17kB 1 2 3 4 5 6 ... 4213 4214 4215 4216 4217 4218
  * NN              (NN) int32 12B 1 2 3
  * teselas         (teselas) int32 4kB 1 2 3 4 5 6 ... 928 929 930 931 932 933
  * Ndir            (Ndir) float64 192B 1.0 2.0 3.0 4.0 ... 21.0 22.0 23.0 24.0
  * Node            (Node) int32 2kB 1 2 3 4 5 6 7 ... 592 593 594 595 596 597
Data variables: (12/14)
    lon_grid        (M) float64 29kB ...
    lon_grid_360    (M) float64 29kB ...
    lat_grid        (N) float64 34kB ...
    lon_node        (Node) float64 5kB ...
    lat_node        (Node) float64 5kB ...
    node_triangle   (teselas, NN) int32 11kB ...
    ...              ...
    simul_hours     int32 4B ...
    simul_dt_hours  int32 4B ...
    Wspeed          int32 4B ...
    ciudad          object 8B ...
    CD_ref          object 8B ...
    dt_forz         int32 4B ...

In [6]:
Dir

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [4]:
import numpy as np
from datetime import datetime, timedelta


dir_steps = len(info_ds.Ndir.values)
N_Tsel = len(info_ds.teselas.values)
Dir = np.arange(dir_steps)
Tes = np.arange(N_Tsel)
DDir, TTes = np.meshgrid(Dir, Tes)
DDir_flat = DDir.flatten()
TTes_flat = TTes.flatten()

date_ref = datetime(*info_ds.date_ref.values)
date_fin = date_ref + timedelta(hours=info_ds.simul_hours.values.item())

metamodel_parameters = {
    "direction": DDir_flat,
    "tesela": TTes_flat,
}

fixed_parameters = {
    "MapInterval": 3600 * info_ds.simul_dt_hours.values,
    "RefDate": date_ref.strftime("%Y%m%d"),
    "StartDateTime": date_ref.strftime("%Y%m%d%H%M%S"),
    "StopDateTime": date_fin.strftime("%Y%m%d%H%M%S"),
    # "CDa": info_ds.CD_Wl_abc.values[0],
    # "CDb": info_ds.CD_Wl_abc.values[1],
    # "VCDa": info_ds.Wl_abc.values[0],
    # "VCDb": info_ds.Wl_abc.values[1],
    # "eddy": info_ds.Eddy.values,
    # "chezy": info_ds.Chezy.values,
    "NetFile": "mesh_Tonga_net.nc",
    "W2Map": 1,
    "dt_forz": info_ds.dt_forz.values,
    "ds_GFD_info": info_ds,
    "wind_magnitude": 40.0,
    "simul_time": 26,
    "dir_steps": dir_steps,
    "grid_nc_file": "outputs/mesh_Tonga_net.nc",
}

In [3]:
metamodel_parameters

{'direction': array([0, 1, 0, 1]), 'tesela': array([0, 0, 1, 1])}

In [4]:
fixed_parameters

{'MapInterval': np.int32(3600),
 'RefDate': '20220101',
 'StartDateTime': '20220101000000',
 'StopDateTime': '20220103000000',
 'NetFile': 'mesh_Tonga_net.nc',
 'W2Map': 1,
 'dt_forz': array(1, dtype=int32),
 'ds_GFD_info': <xarray.Dataset> Size: 149kB
 Dimensions:         (M: 3572, N: 4218, Node: 597, teselas: 933, NN: 3,
                      Ndir: 24, date_ref: 6)
 Coordinates:
   * date_ref        (date_ref) int32 24B 2022 1 1 0 0 0
   * M               (M) int32 14kB 1 2 3 4 5 6 ... 3567 3568 3569 3570 3571 3572
   * N               (N) int32 17kB 1 2 3 4 5 6 ... 4213 4214 4215 4216 4217 4218
   * NN              (NN) int32 12B 1 2 3
   * teselas         (teselas) int32 4kB 1 2 3 4 5 6 ... 928 929 930 931 932 933
   * Ndir            (Ndir) float64 192B 1.0 2.0 3.0 4.0 ... 21.0 22.0 23.0 24.0
   * Node            (Node) int32 2kB 1 2 3 4 5 6 7 ... 592 593 594 595 596 597
 Data variables: (12/14)
     lon_grid        (M) float64 29kB ...
     lon_grid_360    (M) float64 29kB ...
  

In [5]:
import os
from bluemath_tk.wrappers.delft3d.delft3d_wrapper import GreenSurgeModelWrapper


delft3d_wrapper = GreenSurgeModelWrapper(
    templates_dir="/lustre/geocean/WORK/users/faugeree/GreenSurge/Mar_del_Norte_v2/Eje_01/Caso/",
    metamodel_parameters=metamodel_parameters,
    fixed_parameters=fixed_parameters,
    output_dir=os.path.join(os.getcwd(), "Etti_CASES"),
    templates_name=[
        "dimr_config.xml",
        "GreenSurge_GFDcase_wind.ext",
        "GreenSurge_GFDcase.mdu",
    ],
)

delft3d_wrapper.build_cases(
    cases_name_format=lambda ctx: f"GF_T_{ctx.get('tesela')}_D_{ctx.get('direction')}",
)

2025-05-05 13:23:27,909 - GreenSurgeModelWrapper - WARNING - Parameter direction is not in the default_parameters
2025-05-05 13:23:27,910 - GreenSurgeModelWrapper - WARNING - Parameter tesela is not in the default_parameters


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/mesh_Tonga_net.nc'

In [6]:
# delft3d_wrapper.run_cases(launcher="docker_serial", num_workers=10)

In [7]:
delft3d_wrapper.monitor_cases(
    dia_file_name="dflowfmoutput/GreenSurge_GFDcase.dia",
    # value_counts="percentage",
)

,Case,Status
0,GF_T_0_D_0,FINISHED
1,GF_T_0_D_1,FINISHED
2,GF_T_0_D_2,FINISHED
3,GF_T_0_D_3,FINISHED
4,GF_T_0_D_4,FINISHED
...,...,...
571,GF_T_23_D_19,FINISHED
572,GF_T_23_D_20,FINISHED
573,GF_T_23_D_21,FINISHED
574,GF_T_23_D_22,FINISHED


In [8]:
all_Dirs, _T_values, _D_values, _post_eje = delft3d_wrapper.postprocess_cases(
    ds_GFD_info=info_ds,
    parallel=False,
)
all_Dirs

<xarray.Dataset> Size: 9GB
Dimensions:        (tes: 24, dir: 24, time: 25, mesh2d_nFaces: 77139)
Coordinates:
    mesh2d_face_x  (mesh2d_nFaces) float64 617kB -175.2 -175.2 ... -175.1 -175.1
    mesh2d_face_y  (mesh2d_nFaces) float64 617kB -21.17 -21.18 ... -20.81 -20.81
  * time           (time) datetime64[ns] 200B 2022-01-01 ... 2022-01-01T12:00:00
  * tes            (tes) int64 192B 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
  * dir            (dir) int64 192B 0 1 2 3 4 5 6 7 ... 16 17 18 19 20 21 22 23
Dimensions without coordinates: mesh2d_nFaces
Data variables:
    mesh2d_s1      (tes, dir, time, mesh2d_nFaces) float64 9GB nan ... 0.002813
Attributes: (12/18)
    institution:               Deltares
    references:                https://www.deltares.nl
    source:                    D-Flow FM 1.2.184.b981bf9c7329808883c693994582...
    history:                   Created on 2025-04-29T07:54:11-0000, D-Flow FM
    date_created:              2025-04-29T07:54:11-0000
    date_modified:             2025-04-29T07:54:11-0000
    ...                        ...
    geospatial_lon_max:        -174.744455622
    geospatial_lon_units:      degrees_east
    time_coverage_start:       2022-01-01T00:00:00+00:00
    time_coverage_end:         2022-01-01T12:00:00+00:00
    time_coverage_duration:    P0000-00-00T12:00:00
    time_coverage_resolution:  P0000-00-00T00:30:00

In [ ]:
all_Dirs.to_netcdf("outputs/superGSlib.nc")